In [2]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
ek.set_app_key("320e7257c23945729695a106cedb2b9cb72d3207")

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

# Economic Story

**TR.RevenueActValue.date** - prints the datetime

## Part 1: Accuracy

1. **TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

2. **TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> this is a analyst forecast variable

3. **TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> forecast error between actual EPS and TR.EPSMean

<img src="../Pictures/Screenshot 2023-02-15 185222.png" width=500 height=300 />

## Part 2: Factors

### Firm charateristics

1. **TR.RevenueActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Revenue (or Sales) is a corporation's net revenue, generally derived from core business activities. For non-financial companies, the calculation of net revenue (or net turnover) in most markets generally involves subtracting transportation and related operational costs from gross revenue/sales. Revenue recognition practices vary significantly from market to market, though generally the recording of revenue is based upon sales invoices issued (or anticipated for forecast purposes) during the accounting period.

2. **TR.F.EV** - Enterprise Value is the Company's total economic value OR the theoretical takeover price, often used as an alternative to market capitalization. Enterprise Value represents Market Capitalization TR.F.MktCap + Debt including Preferred Equity & Minority Interest - Total [TR.F.DebtlnclPrefEqMinlntrTot] (-) Cash & Short Term Investments - Total [TR.F.CashSTlnvstTot]. Market Capitalization [TR.F.MktCap] is required. The data item is calculated for all periodicities. It is applicable to all industries.

3. **TR.LTGMean** - Long Term Growth - Mean:  The statistical average of all broker estimates determined to be on the majority accounting basis. Long-term growth is an estimate of the compound average rate of EPS growth an analyst expects over a period of three to five ears.

### Analysts
1. **TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.

2. **TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).

### Macro characteristics

1. **TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.

### Loss Firm Status (0 or 1)

1. **TR.F.NetIncAfterTax** Net Income after Tax [SIAT] represents the income/expense after all operating and non-operating income and expense, reserves, income taxes, but before equity in earnings, minority interest, extraordinary items, after-tax adjustments, discontinued operations and preferred dividends. Applicable to all Industries. Net Income after Tax [SIAT] includes: • Net Income after Tax [XIAT]

### More difficult Factors (yearly):

#### ESG

1. **TR.TRESGScore** Refinitiv ESG Score is an overall company score based on the self-reported information in the environmental, social and corporate governance pillars.

#### Proportion of independent Directors

1. **TR.AnalyticIndepBoard** - Independent Board Members - Percentage of independent board members as reported by the company.

2. **TR.GovernancePillarScore** - Governance Pillar Score - The corporate governance pillar measures a company's systems and processes, which ensure that its board members and executives act in the best interests of its long term shareholders. It reflects a company's capacity, through its use of best management practices, to direct and control its rights and responsibilities through the creation of incentives, as well as checks and balances in order to generate long term shareholder value.

## Part 3: Regression

### continue..

In [3]:
economic_story = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise", 'TR.RevenueActValue', "TR.F.EV", "TR.LTGMean",  "TR.NumberOfAnalysts", "TR.RecMean", "TR.RecMean(SDate=-2,EDate=-11)", "TR.WACCInflationAdjRiskFreeRate", "TR.F.NetIncAfterTax"]
# , "TR.AdjSharesHeld"

In [4]:
df, e = ek.get_data('0#.SPX', economic_story, parameters = {'SDate':'-1','EDate':'-10','Period':'FQ0','Frq':'FQ'})
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000,14061600372.299999,<NA>,10,2.27273,2.2,3.747421,189965000
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000,15561870705.290001,<NA>,9,2.2,2.0,3.092855,307205000
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000,18436823034.200001,<NA>,9,2.0,2.3,2.325202,179203000
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000,23848820586.0,<NA>,8,2.3,2.3,1.515266,107542000
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000,17699543822.66,<NA>,8,2.3,2.3,1.527139,184573000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,AVY.N,2021-07-28 06:45:00+00:00,2.25,2.05744,9.359,2102000000,19132322317.099998,9.4,11,2.15385,2.14286,1.444323,184900000
5026,AVY.N,2021-04-28 06:45:00+00:00,2.4,2.0288,18.297,2051300000,17428269089.919998,7.844,11,2.14286,2.21429,1.745787,210800000
5027,AVY.N,2021-02-03 06:45:00+00:00,2.27,2.08642,8.799,1990900000,14803422859.42,8.933,12,2.21429,2.35714,0.926432,192400000
5028,AVY.N,2020-10-21 06:45:00+00:00,1.91,1.52673,25.104,1729100000,11816435091.5,7.958,12,2.35714,2.42857,0.677434,151500000


In [5]:
cols = []
count = 1
for column in df.columns:
    if column == 'Recommendation - Mean (1-5)':
        cols.append(f'Recommendation - Mean (1-5)_{count}')
        count+=1
        continue
    cols.append(column)
df.columns = cols
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5)_1,Recommendation - Mean (1-5)_2,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000,14061600372.299999,<NA>,10,2.27273,2.2,3.747421,189965000
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000,15561870705.290001,<NA>,9,2.2,2.0,3.092855,307205000
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000,18436823034.200001,<NA>,9,2.0,2.3,2.325202,179203000
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000,23848820586.0,<NA>,8,2.3,2.3,1.515266,107542000
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000,17699543822.66,<NA>,8,2.3,2.3,1.527139,184573000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,AVY.N,2021-07-28 06:45:00+00:00,2.25,2.05744,9.359,2102000000,19132322317.099998,9.4,11,2.15385,2.14286,1.444323,184900000
5026,AVY.N,2021-04-28 06:45:00+00:00,2.4,2.0288,18.297,2051300000,17428269089.919998,7.844,11,2.14286,2.21429,1.745787,210800000
5027,AVY.N,2021-02-03 06:45:00+00:00,2.27,2.08642,8.799,1990900000,14803422859.42,8.933,12,2.21429,2.35714,0.926432,192400000
5028,AVY.N,2020-10-21 06:45:00+00:00,1.91,1.52673,25.104,1729100000,11816435091.5,7.958,12,2.35714,2.42857,0.677434,151500000


In [6]:
df["Recommendation change"] = df["Recommendation - Mean (1-5)_1"] - df["Recommendation - Mean (1-5)_2"]
df.insert(10, "Recommendation change", df.pop("Recommendation change"))
df = df.rename(columns={"Recommendation - Mean (1-5)_1": "Recommendation - Mean (1-5)"})
df = df.drop("Recommendation - Mean (1-5)_2", axis=1)
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax
0,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196,1615339000,14061600372.299999,<NA>,10,2.27273,0.07273,3.747421,189965000
1,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503,2055818000,15561870705.290001,<NA>,9,2.2,0.2,3.092855,307205000
2,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342,1412650000,18436823034.200001,<NA>,9,2.0,-0.3,2.325202,179203000
3,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267,1035557000,23848820586.0,<NA>,8,2.3,0.0,1.515266,107542000
4,POOL.OQ,2021-10-21 07:00:00+00:00,4.51,3.84833,17.194,1411448000,17699543822.66,<NA>,8,2.3,0.0,1.527139,184573000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,AVY.N,2021-07-28 06:45:00+00:00,2.25,2.05744,9.359,2102000000,19132322317.099998,9.4,11,2.15385,0.01099,1.444323,184900000
5026,AVY.N,2021-04-28 06:45:00+00:00,2.4,2.0288,18.297,2051300000,17428269089.919998,7.844,11,2.14286,-0.07143,1.745787,210800000
5027,AVY.N,2021-02-03 06:45:00+00:00,2.27,2.08642,8.799,1990900000,14803422859.42,8.933,12,2.21429,-0.14285,0.926432,192400000
5028,AVY.N,2020-10-21 06:45:00+00:00,1.91,1.52673,25.104,1729100000,11816435091.5,7.958,12,2.35714,-0.07143,0.677434,151500000


In [7]:
df['New Date'] = pd.Series()

for i in range(len(df)):
    if df.loc[i, "Date"].month == 1:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 2:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 3:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=1, day=1)
    elif df.loc[i, "Date"].month == 4:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 5:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 6:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=4, day=1)
    elif df.loc[i, "Date"].month == 7:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 8:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 9:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=7, day=1)
    elif df.loc[i, "Date"].month == 10:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 11:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)
    elif df.loc[i, "Date"].month == 12:
        df.loc[i, "New Date"] = df.loc[i, "Date"].replace(month=10, day=1)

df.insert(2, "New Date", df.pop("New Date"))
df["New Date"] = pd.to_datetime(df["New Date"]).dt.date

df

/var/folders/g8/1tyvls5x41bd_dx0zhcy5ggc0000gn/T/ipykernel_14562/2145643216.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['New Date'] = pd.Series()


,Instrument,Date,New Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax
0,POOL.OQ,2022-10-20 07:00:00+00:00,2022-10-01,4.78,4.5875,4.196,1615339000,14061600372.299999,<NA>,10,2.27273,0.07273,3.747421,189965000
1,POOL.OQ,2022-07-21 07:00:00+00:00,2022-07-01,7.63,7.517,1.503,2055818000,15561870705.290001,<NA>,9,2.2,0.2,3.092855,307205000
2,POOL.OQ,2022-04-21 07:00:00+00:00,2022-04-01,4.23,3.14867,34.342,1412650000,18436823034.200001,<NA>,9,2.0,-0.3,2.325202,179203000
3,POOL.OQ,2022-02-17 07:00:00+00:00,2022-01-01,2.63,1.875,40.267,1035557000,23848820586.0,<NA>,8,2.3,0.0,1.515266,107542000
4,POOL.OQ,2021-10-21 07:00:00+00:00,2021-10-01,4.51,3.84833,17.194,1411448000,17699543822.66,<NA>,8,2.3,0.0,1.527139,184573000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,AVY.N,2021-07-28 06:45:00+00:00,2021-07-01,2.25,2.05744,9.359,2102000000,19132322317.099998,9.4,11,2.15385,0.01099,1.444323,184900000
5026,AVY.N,2021-04-28 06:45:00+00:00,2021-04-01,2.4,2.0288,18.297,2051300000,17428269089.919998,7.844,11,2.14286,-0.07143,1.745787,210800000
5027,AVY.N,2021-02-03 06:45:00+00:00,2021-01-01,2.27,2.08642,8.799,1990900000,14803422859.42,8.933,12,2.21429,-0.14285,0.926432,192400000
5028,AVY.N,2020-10-21 06:45:00+00:00,2020-10-01,1.91,1.52673,25.104,1729100000,11816435091.5,7.958,12,2.35714,-0.07143,0.677434,151500000


In [8]:
economic_story_yearly = ["TR.TRESGScore.date", "TR.TRESGScore", "TR.GovernancePillarScore", "TR.AnalyticIndepBoard"]

In [9]:
df2, e = ek.get_data('0#.SPX', economic_story_yearly, parameters = {'SDate':'0','EDate':'-5','Period':'FY0','Frq':'FY'})
df2["Date"] = pd.to_datetime(df2["Date"])
df2

,Instrument,Date,ESG Score,Governance Pillar Score,Independent Board Members
0,POOL.OQ,2021-12-31 00:00:00+00:00,48.718747,79.856229,80.0
1,POOL.OQ,2020-12-31 00:00:00+00:00,40.485976,69.05684,88.888889
2,POOL.OQ,2019-12-31 00:00:00+00:00,44.618552,80.983486,88.888889
3,POOL.OQ,2018-12-31 00:00:00+00:00,42.83681,66.013223,87.5
4,POOL.OQ,2017-12-31 00:00:00+00:00,39.816484,54.70234,75.0
...,...,...,...,...,...
3000,AVY.N,2021-01-02 00:00:00+00:00,59.629421,30.056165,90.909091
3001,AVY.N,2019-12-28 00:00:00+00:00,54.240166,21.795439,83.333333
3002,AVY.N,2018-12-29 00:00:00+00:00,52.999907,40.477426,83.333333
3003,AVY.N,2017-12-30 00:00:00+00:00,55.439872,50.729392,81.818182


In [10]:
df['ESG Score'] = pd.Series()
df['Governance Pillar Score'] = pd.Series()
df['Independent Board Members'] = pd.Series()

for i in range(len(df2)):
    for j in range(len(df)):
        if df.loc[j, "Instrument"] == df2.loc[i, "Instrument"] and df.loc[j, "New Date"].year == df2.loc[i, "Date"].year:
            df.loc[j, "ESG Score"] = df2.loc[i, "ESG Score"]
            df.loc[j, "Governance Pillar Score"] = df2.loc[i, "Governance Pillar Score"]
            df.loc[j, "Independent Board Members"] = df2.loc[i, "Independent Board Members"]

df

/var/folders/g8/1tyvls5x41bd_dx0zhcy5ggc0000gn/T/ipykernel_14562/408457973.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['ESG Score'] = pd.Series()
/var/folders/g8/1tyvls5x41bd_dx0zhcy5ggc0000gn/T/ipykernel_14562/408457973.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Governance Pillar Score'] = pd.Series()
/var/folders/g8/1tyvls5x41bd_dx0zhcy5ggc0000gn/T/ipykernel_14562/408457973.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Independent Board Members'] = pd.Series()


,Instrument,Date,New Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Long Term Growth - Mean,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,ESG Score,Governance Pillar Score,Independent Board Members
0,POOL.OQ,2022-10-20 07:00:00+00:00,2022-10-01,4.78,4.5875,4.196,1615339000,14061600372.299999,<NA>,10,2.27273,0.07273,3.747421,189965000,NaN,NaN,NaN
1,POOL.OQ,2022-07-21 07:00:00+00:00,2022-07-01,7.63,7.517,1.503,2055818000,15561870705.290001,<NA>,9,2.2,0.2,3.092855,307205000,NaN,NaN,NaN
2,POOL.OQ,2022-04-21 07:00:00+00:00,2022-04-01,4.23,3.14867,34.342,1412650000,18436823034.200001,<NA>,9,2.0,-0.3,2.325202,179203000,NaN,NaN,NaN
3,POOL.OQ,2022-02-17 07:00:00+00:00,2022-01-01,2.63,1.875,40.267,1035557000,23848820586.0,<NA>,8,2.3,0.0,1.515266,107542000,NaN,NaN,NaN
4,POOL.OQ,2021-10-21 07:00:00+00:00,2021-10-01,4.51,3.84833,17.194,1411448000,17699543822.66,<NA>,8,2.3,0.0,1.527139,184573000,48.718747,79.856229,80.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,AVY.N,2021-07-28 06:45:00+00:00,2021-07-01,2.25,2.05744,9.359,2102000000,19132322317.099998,9.4,11,2.15385,0.01099,1.444323,184900000,59.629421,30.056165,90.909091
5026,AVY.N,2021-04-28 06:45:00+00:00,2021-04-01,2.4,2.0288,18.297,2051300000,17428269089.919998,7.844,11,2.14286,-0.07143,1.745787,210800000,59.629421,30.056165,90.909091
5027,AVY.N,2021-02-03 06:45:00+00:00,2021-01-01,2.27,2.08642,8.799,1990900000,14803422859.42,8.933,12,2.21429,-0.14285,0.926432,192400000,59.629421,30.056165,90.909091
5028,AVY.N,2020-10-21 06:45:00+00:00,2020-10-01,1.91,1.52673,25.104,1729100000,11816435091.5,7.958,12,2.35714,-0.07143,0.677434,151500000,NaN,NaN,NaN


In [11]:
df.dtypes

Instrument                                                  string
Date                                           datetime64[ns, UTC]
New Date                                                    object
Earnings Per Share - Actual                                Float64
Earnings Per Share - Mean                                  Float64
Earnings Per Share - Actual Surprise                       Float64
Revenue - Actual                                             Int64
Enterprise Value                                           Float64
Long Term Growth - Mean                                    Float64
Number of Analysts                                           Int64
Recommendation - Mean (1-5)                                Float64
Recommendation change                                      Float64
WACC Inflation Adjusted Risk Free Rate, (%)                Float64
Net Income after Tax                                         Int64
ESG Score                                                  flo